In [1]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
#set local path for GTEX data and supporting txt files
ROOT_PATH = "C:\\Users\\micha\\Downloads\\"

In [4]:
#total number of transcripts is about 56K
#total number of samples is about 15K
#experiment with various subsets
num_of_genes = 500
num_of_samples = 2000

In [5]:
#order of samples in this file defines sample id
sample_columns = os.path.join(ROOT_PATH, "Sample_Columns.txt")
sample_file = open(sample_columns,'r')
sample_names = []
for line in sample_file:
    parts = line.split('\t')
    for part in parts:
        sample_name = part.rstrip()
        sample_names.append(sample_name)

#remove 'Name' and 'Description' from sample_names
del(sample_names[0])
del(sample_names[0])

#create sample name->local id dictionary for later use
sample_to_id = {}
sample_id = 1

for name in sample_names:
    sample_to_id[name] = sample_id
    sample_id = sample_id + 1

In [6]:
#keep track sample id to tissue id in dictionary
sample_to_tissue = {}
tissue_to_id = {}
tissues = []
sample_tissue_path = os.path.join(ROOT_PATH, "Sample_to_Tissue.txt")
sample_tissue_file = open(sample_tissue_path,'r')
#local tissue id starting with 1 is based on order listed in Sample_to_Tissue.txt
tissue_id = 1
for line in sample_tissue_file:
    parts = line.split('\t')
    sample_name = parts[0].rstrip()
    if (sample_name in sample_to_id.keys()):
        sample_id = sample_to_id[sample_name]
        tissue = parts[1].rstrip()
        tissue = tissue.replace(',', ' ')
        if (tissue not in tissue_to_id.keys()):
            tissue_to_id[tissue] = tissue_id
            tissues.append(tissue)
            tissue_id = tissue_id + 1
        sample_to_tissue[sample_id] = tissue_to_id[tissue]

In [7]:
#for the load_csv_with_header method used below
# the header in the training and test files needs to be a comma-separated
#list of number of entries, number of features, followed by list of all possible classes/categories
header_string = str(num_of_samples) + "," + str(num_of_genes) +"," + ",".join(tissues)

In [8]:
#number of classes used as argument to classifier below
num_of_classes = len(tissues)

In [9]:
tpm_data = os.path.join(ROOT_PATH, "GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct")
#skip the first 2 rows
#in the reader the columns are the samples, rows are the genes
#the matrix will be transposed in the csv file created below
reader = pd.read_table(tpm_data, sep = "\t",skiprows = [0,1], nrows=num_of_genes)

In [10]:
#update the version of the csv file to experiment with different training and test sets
TRAINING = "training7.csv"
TEST = "test7.csv"

In [11]:
#need to reformat data for tf's load_csv_with_header
#f is the file handle for the training data set
f = open(TRAINING, "w")
f.write(header_string + "\n")
#tpm values for sample1 begins in column 3 of the reader so the column index, "i" is initially set to 2 
i = 2
sample_id = 1
while i < 2 + num_of_samples:
    #temp list of tpm values for a given sample
    list = []
    tissue_id = sample_to_tissue[sample_id] 
    #each column contains expression values for each gene in a sample
    for index, column in reader.iterrows() :
        list.append(str(round(column[i],0)))
    #need to add tissue id as the class/target value for the NN
    #append tissue_id to end of line
    list.append(str(tissue_id))
    line = ",".join(list) + "\n"
    f.write(line)
    #use odd-numbered samples for the training set
    #even-numbered samples will be used in the test set
    sample_id =  sample_id + 2
    #increment i by 2 to get corresponding tpm values for each odd-numbered sample
    i = i + 2
f.close()

In [12]:
#test data set will use even-numbered samples from the reader
f_test = open(TEST, "w")
f_test.write(header_string + "\n")
#column index for sample2 is 3
i = 3
sample_id = 2
while i < 2 + num_of_samples:
    list = []
    tissue_id = sample_to_tissue[sample_id] 
    #each column contains expression values for each gene in a sample
    for index, column in reader.iterrows() :
        list.append(str(round(column[i],0)))
    #append tissue_id to end of line, it will be used as the class/target used by NN classifier
    list.append(str(tissue_id))
    line = ",".join(list) + "\n"
    f_test.write(line)
    sample_id = sample_id + 2
    i = i + 2
f_test.close()

In [13]:
# load datasets
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=TRAINING,
      target_dtype=np.int,
      features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=TEST,
      target_dtype=np.int,
      features_dtype=np.float32)

In [14]:
# Specify that all features have real-value data
feature_columns = [tf.feature_column.numeric_column("x", shape=[num_of_genes])]

In [15]:
# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                          hidden_units=[10, 20, 10],
                                          n_classes=num_of_classes + 1,
                                          model_dir="")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\micha\\AppData\\Local\\Temp\\tmp_srzdq6m', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [16]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(training_set.data)},
      y=np.array(training_set.target),
      num_epochs=None,
      shuffle=True)

In [17]:
# Train model.
classifier.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\micha\AppData\Local\Temp\tmp_srzdq6m\model.ckpt.
INFO:tensorflow:loss = 7489.19, step = 1
INFO:tensorflow:global_step/sec: 457.26
INFO:tensorflow:loss = 350.742, step = 101 (0.219 sec)
INFO:tensorflow:global_step/sec: 533.279
INFO:tensorflow:loss = 333.916, step = 201 (0.188 sec)
INFO:tensorflow:global_step/sec: 533.276
INFO:tensorflow:loss = 277.266, step = 301 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.237
INFO:tensorflow:loss = 289.969, step = 401 (0.188 sec)
INFO:tensorflow:global_step/sec: 492.271
INFO:tensorflow:loss = 263.577, step = 501 (0.203 sec)
INFO:tensorflow:global_step/sec: 533.276
INFO:tensorflow:loss = 222.223, step = 601 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.26
INFO:tensorflow:loss = 289.181, step = 701 (0.188 sec)
INFO:tensorflow:global_step/sec: 533.227
INFO:tensorflow:loss = 211.298, step = 801 (0.188 sec)
INFO:tensorflow:global_step/sec: 533.325
INFO

In [18]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(test_set.data)},
      y=np.array(test_set.target),
      num_epochs=1,
      shuffle=False)

In [19]:
# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

INFO:tensorflow:Starting evaluation at 2017-10-25-05:00:08
INFO:tensorflow:Restoring parameters from C:\Users\micha\AppData\Local\Temp\tmp_srzdq6m\model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2017-10-25-05:00:08
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.5, average_loss = 1.72207, global_step = 2000, loss = 215.259

Test Accuracy: 0.500000



In [ ]:
#load in median tpm values for each tissue type
#test calssifier against median values

In [ ]:
# Classify one new tissue sample.

new_samples = np.array(
#test example with 100 genes - need to update if number of included genes change
    [[0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,322.0,30.0,9.0,7.0,2306.0,9.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,62.0,12.0,1.0,0.0,8.0,46.0,0.0,0.0,82.0,18.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,235.0,16.0,1.0,0.0,24.0,3.0,2.0,29.0,9.0,61.0,28.0,17.0,0.0,62.0]], dtype=np.float32)
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": new_samples},
      num_epochs=1,
      shuffle=False)

predictions = classifier.predict(input_fn=predict_input_fn)
#dir(predictions)
d = next(predictions)
print(type(d))
print(d.keys())
probs= d['probabilities']
max_p_index = np.argmax(probs)
print(max_p_index)
print(tissues[max_p_index])